In [1]:
import pandas as pd
import numpy as np
import re
import time
import datetime
import requests as rq

import youtube_dl 
from tqdm import tqdm_notebook

pd.set_option("display.max_columns", None)

## Coleta

- Dados foram coletados no arquivo `coleta_videos.py`

In [2]:
df_videos = pd.read_json("parsed_videos.json", lines=True)
df_videos.head()

,link,title,query
0,/watch?v=LkPmTGw1jqo,S0-Introduction-Module3: Deep Learning and AI ...,machine+learning
1,/watch?v=R_gBq8IfwJc,Machine Learning no Ensino Médio,machine+learning
2,/watch?v=Tix2xon9MSs,NBA LIVE 22: EA COULD USE DEEP MACHINE LEARNIN...,machine+learning
3,/watch?v=w-2ip78NxAw,Amazon Aurora Machine Learning – SageMaker Int...,machine+learning
4,/watch?v=f6XIY_M7FlA,Machine Learning and Deep Learning Implementat...,machine+learning


In [3]:
df_videos.shape

(1695, 3)

### Verificação dos Dados

In [4]:
URL = "https://www.youtube.com{link}"

In [5]:
ydl = youtube_dl.YoutubeDL({"ignoreerrors": True, 'verbose':False})

In [6]:
video_list = []
for link in tqdm_notebook(df_videos['link'].unique()):

    video_info  = {}
    try:
        r = ydl.extract_info(url=URL.format(link=link), download=False)
        year = r['upload_date'][:4]
        month = r['upload_date'][4:6]
        day = r['upload_date'][6:]
    except Exception:
        print(f"Fail - {link}")
        continue
    
    video_info = {
            'uploader': r['uploader'],
            'title': r['title'],
            'upload_date': f"{year}-{month}-{day}",
            'user': r['uploader_id'],
            'view_count': r['view_count'],
            'like_count': r['like_count'],
            'dislike_count': r['dislike_count'],
            'thumbnail': r['thumbnail'],
            'width': r['width'],
            'height': r['height'],
            'categories': '|'.join(r['categories']) if r['categories'] is not None else None,
            'tags': '|'.join(r['tags']) if r['tags'] is not None else None,
            'channel_url': r['channel_url'],
            'description': r['description']
            }
    video_list.append(video_info)
    

/home/cesar/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


[youtube] LkPmTGw1jqo: Downloading webpage
[youtube] LkPmTGw1jqo: Downloading MPD manifest
[youtube] R_gBq8IfwJc: Downloading webpage


ERROR: R_gBq8IfwJc: YouTube said: Unable to extract video data


Fail - /watch?v=R_gBq8IfwJc
[youtube] Tix2xon9MSs: Downloading webpage


ERROR: Tix2xon9MSs: YouTube said: Unable to extract video data


...

ERROR: w-2ip78NxAw: YouTube said: Unable to extract video data


Fail - /watch?v=w-2ip78NxAw
[youtube] f6XIY_M7FlA: Downloading webpage


ERROR: f6XIY_M7FlA: YouTube said: Unable to extract video data


Fail - /watch?v=f6XIY_M7FlA
[youtube] B2YupcaJcWw: Downloading webpage



KeyboardInterrupt: 

In [ ]:
df_video_info = pd.DataFrame(video_list)
df_video_info.to_csv("raw_data_sem_labels.csv")